In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
%env CUDA_VISIBLE_DEVICES=7

env: CUDA_VISIBLE_DEVICES=7


## Выбор кейса 

В данной работе я ставлю перед собой следущую бизнес-цель: решить задачу **бинарной классификации вакансий на настоящие и мошеннические** для антифрод-системы сервиса с объявлениями о работе. В данной задаче классы сильно несбалансированны, так как мошеннических вакансий обычно существенно меньше, чем настоящих.

## Сбор данных

### Датасет для исходной задачи

Для своей задачи я нашел два подходящих датасет [Real / Fake Job Posting Prediction](https://www.kaggle.com/datasets/shivamb/real-or-fake-fake-jobposting-prediction) с 18 тыс. вакансий, из которых около 800 являются фейковыми

Датасет имеет лицензию [CC0: Public Domain](https://creativecommons.org/publicdomain/zero/1.0/), поэтому его можно использовать для обучающих, исследовательских и коммерческих целей, не спрашивая разрешения у авторов.

Загрузим датасет и прочитаем его

In [ ]:
# !mkdir data/real_fake_postings/ && cd data/real_fake_postings/ && kaggle datasets download shivamb/real-or-fake-fake-jobposting-prediction && unzip real-or-fake-fake-jobposting-prediction.zip 

In [ ]:
real_fake_df = pd.read_csv("data/real_fake_postings/fake_job_postings.csv")

In [5]:
real_fake_df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


### Weak Supervision

#### Датасет из смежной задачи
Из смежной задачи, которая заключается в предсказании зарплаты по данным вакансии, я нашёл датасет [LinkedIn Job Postings (2023 - 2024)](https://www.kaggle.com/datasets/arshkon/linkedin-job-postings/data) с более 100 тыс. вакансиями из LinkedIn. Датасет имеет лицензию [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/), поэтому его можно использовать для исследовательских и коммерческих целей, но выпуская наш продукт под той же лицензией с указанием ссылки на исходный датасет.

В этом датасете нет информации о том, является ли заданное объявляение фейком или нет, но я воспользуюсь Weak Supervision, чтобы доразметить его. В частности, я хочу излвечь оттуда явно фейковые вакансии, что поможет увеличить датасет и уменьшить дисбаланс классов.


#### Подход для доразметки
Я буду помечать вакансию как фейковую, если в её описании встерчаются определенные фразы. Например, "No experience required", "Unlimited earning potential", "Financial freedom in weeks". После чего из таких вакансий выберу top-N с самыми короткими описаниями профилей компаний (фейковые компании обычно имеют не очень развернутые описания).

Загрузим датасет и найтем те объявляения, которые, скорее всего, являются фейковыми

In [ ]:
unlabeled_postings = pd.read_csv("data/linkedin_postings/postings.csv")
companies = pd.read_csv("data/linkedin_postings/companies/companies.csv")[["company_id", "description"]]
unlabeled_postings = pd.merge(left=unlabeled_postings, right=companies, how="left", on="company_id", suffixes=["", "_company"])
unlabeled_postings.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips,description_company
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0,With years of experience helping local buyers ...
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0,NaN
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0,"In April of 1983, The National Exemplar began ..."
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0,"Abrams Fensterman, LLP is a full-service law f..."
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0,NaN


In [ ]:
import re

def is_fraudulent_job_description(text):
    red_flag_patterns = [
        r'\b(earn\s*\$\d+[\d,.]*\s*(per|/)\s*(month|week|hour|yr|year)|make \$[\d,.]+\s+(fast|quick))\b',
        r'\b(pay\s*(a|the)\s+fee|upfront\s+cost|security\s+deposit|required\s+investment)\b',
        r'\b(no\s+experience\s+required|no\s+qualifications\s+needed)\b',
        r'\b(guaranteed\s+income|financial\s+freedom\s+in\s+\d+\s+weeks)\b',
        r'\b(multi[\s-]*level\s+marketing|mlm|pyramid\s+scheme)\b',
        r'\b(send\s+(your\s+)?(personal\s+)?(information|details|bank\s+account|ssn|social\s+security))\b',
        r'\b(work\s+from\s+home\s+(with\s+)?no\s+(experience|interview))\b',
        r'\b(cryptocurrency\s+investment|bitcoin\s+mining|process\s+payments)\b',
        r'\b(recruit\s+\d+\s+people|build\s+your\s+team|referral\s+commissions)\b',
        r'\b(urgently\s+hiring|immediate\s+start|positions?\s+available\s+now)\b',
        r'\b(kindly\s+send|dear\s+candidate,|this\s+is\s+not\s+a\s+scam)\b',
        r'\b(free\s+training\s+materials|company\s+will\s+send\s+you\s+a\s+check)\b',
        r'\b(government\s+approved|100%\s+legitimate|risk-free\s+opportunity)\b',
        r'\b(wire\s+transfers|international\s+transactions|money\s+transfer)\b',
    ]

    combined_pattern = re.compile(
        '(' + '|'.join(red_flag_patterns) + ')',
        flags=re.IGNORECASE
    )

    return bool(combined_pattern.search(text))

In [ ]:
is_fraud_job_desc = unlabeled_postings.description.fillna("").apply(lambda desc: is_fraudulent_job_description(desc))

suspicious_jobs = unlabeled_postings[is_fraud_job_desc]
suspicious_jobs.loc[:, "description_company_len"] = unlabeled_postings[is_fraud_job_desc].description_company.fillna("").apply(lambda desc: len(desc))

suspicious_jobs = suspicious_jobs[suspicious_jobs.company_id.isna() | ~suspicious_jobs.company_id.duplicated()]
topN = 300
suspicious_jobs_idx = suspicious_jobs.description_company_len.sort_values(ascending=True)[:topN].index
suspicious_jobs_final = suspicious_jobs.loc[suspicious_jobs_idx]

In [209]:
suspicious_jobs_final.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips,description_company,description_company_len
23561,3889766866,NaN,Vice President Finance,Reports To: CEOFLSA: ExemptLocation: Remote Wh...,200000.0,YEARLY,"Shirley, NY",NaN,33.0,NaN,...,NaN,0,FULL_TIME,USD,BASE_SALARY,190000.0,11967.0,36103.0,NaN,0
23618,3889770622,NaN,Treasury Analyst,Overview of the Position:\nFull time position ...,NaN,NaN,New York City Metropolitan Area,NaN,11.0,NaN,...,NaN,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN,NaN,0
23302,3889751726,NaN,Gift Shop Manager,Gift Shop Manager The White House Historical A...,65000.0,YEARLY,"Washington, DC",NaN,6.0,NaN,...,NaN,0,FULL_TIME,USD,BASE_SALARY,60000.0,20001.0,11001.0,NaN,0
36952,3895599731,A Hiring Company,Kitchen Leader,Go Chicken Go Hiring Now!\n\n \n\nAt Go Chicke...,NaN,NaN,"Missouri City, TX",101478385.0,2.0,NaN,...,www.click2apply.net,0,FULL_TIME,NaN,NaN,NaN,77459.0,48157.0,NaN,0
118020,3906085362,NaN,Unemployed,Remote Positions Available!!!\nFinancial offic...,NaN,NaN,United States,NaN,11.0,NaN,...,NaN,0,OTHER,NaN,NaN,NaN,NaN,NaN,NaN,0


### Метрика качества и тестовый датасет
#### Метрика качества
Наша задача -- бинарная классификация с дисбалансом классов, поэтому выберем интепретируемые метрики, которые устойчивы к дисбалансу классов:
1. F1-score
2. Precision
3. Recall

Отметим, что мы не выбрали AUC-ROC по причине того, что в задачах, где не так важен больший класс, он может давать не совсем адекватную картину при сравнении алгоритмов.

В нашей задаче важен как precision (не хотим банить настоящие объявления), так и recall (не хотим пропускать мошеннические объявления), поэтому в качестве основной метрики возьмем **f1-score**, так как она балансирует между предыдущими двумя. Precision и Recall будут второстепенными метриками.

#### Разбиение датасета на обучающий и тестовый
Мы могли бы разбить на трейн и тест случайно. Однако, мы воспользуемся стратифицированным по таргету разбиением, потому что это уменьшит разброс долей положительного класса между обучащей и тестовой выборками.

In [212]:
from sklearn.model_selection import train_test_split
y = real_fake_df.fraudulent
X = real_fake_df.drop(columns="fraudulent")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

### Подготовка к обучению